In [1]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
nltk.download("punkt")
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer




[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gokul_08/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/gokul_08/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def Web_crawlerFunc(max_pages,output_list,keyword):

    page = 0
    index = 0
    
    while(page <= max_pages):
        URL = ("https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=" + str(page))
        page += 1
        #print("fetching " + URL)
        source_code = requests.get(URL)
#         if URL.status_code != 200:
#             page=99
#             break
        plainText = source_code.text
        soup = BeautifulSoup(plainText, "html.parser")
        
        for data in soup.find_all("div", {"class": "result-container"}):
            data_dict = {}
            if (data != None ):
                index += 1
                flag = 1
                #print(index)
                data_dict["Index"] = index
                data_dict["Author Link"] = data.find('a' , {"class" : "link person"}).get('href') if data.find('a' , {"class" : "link person"})!= None else "NA"
                data_dict["Title Link"] = data.find('a' , {"class" : "link"}).get('href') if data.find('a' , {"class" : "link"})!= None else "NA"
                data_dict["Author Name"] = data.find('a' , {"class" : "link person"}).get_text() if data.find('a' , {"class" : "link person"})!= None else "NA"
                data_dict["Year of publication"] = data.find('span' , {"class" : "date"}).get_text() if data.find('span' , {"class" : "date"})!= None else "NA"
                
                if data_dict.get("Author Link") == "NA" or data_dict.get("Title Link") == "NA" or data_dict.get("Author Name") == "NA" or data_dict.get("Year of publication") == "NA":
                    flag =0
                    data_dict={}
                if flag ==1 :
                    title = data.find('a' , {"class" : "link"}).get_text() + " " + data.find('a' , {"class" : "link person"}).get_text() +" "+ data.find('span' , {"class" : "date"}).get_text()
                    data_dict["Keywords"] = title
                    keyword.append(title)
                    output_list.append(data_dict)
                #print(output_list)
                
                
                

            
                
    print(output_list)

output_list =  []

keyword = []
Web_crawlerFunc(2,output_list,keyword)

[{'Index': 1, 'Author Link': 'https://pureportal.coventry.ac.uk/en/persons/rashedul-hasan', 'Title Link': 'https://pureportal.coventry.ac.uk/en/publications/a-bibliometric-review-of-the-waqf-literature', 'Author Name': 'Hasan, R.', 'Year of publication': 'Jun 2022', 'Keywords': 'A bibliometric review of the Waqf literature Hasan, R. Jun 2022'}, {'Index': 2, 'Author Link': 'https://pureportal.coventry.ac.uk/en/persons/rashedul-hasan', 'Title Link': 'https://pureportal.coventry.ac.uk/en/publications/a-note-on-covid-19-instigated-maximum-drawdown-in-islamic-markets', 'Author Name': 'Hasan, R.', 'Year of publication': 'May 2022', 'Keywords': 'A note on COVID-19 instigated maximum drawdown in Islamic markets versus conventional counterparts Hasan, R. May 2022'}, {'Index': 3, 'Author Link': 'https://pureportal.coventry.ac.uk/en/persons/alireza-zarei', 'Title Link': 'https://pureportal.coventry.ac.uk/en/publications/bank-stock-valuation-theories-do-they-explain-prices-based-on-the', 'Author N

In [4]:
file = open("keywords.txt", "w", encoding='utf-8')
file.write("")
f = open("keywords.txt", "a", encoding='utf-8')
for i in keyword:
    f.write( "'"+i+"',"+ '\n')
f.close()

In [5]:
f = open('keywords.txt', 'r', encoding='utf-8')
read1 = f.read().split("\n")

file = open('keywords.txt', encoding='utf-8')
read2 = file.read()
   

In [6]:
#remove punctuations

pun = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''  
for x in read2:
    
    #print(x)
    if x in pun:
        read2 = read2.replace(x, " ")


In [7]:
count = 1 
for w in read2:
    if w == '\n':
        count =count +1

In [8]:
#Preprocessing
sw = stopwords.words('english')
#print(sw)
ps = PorterStemmer()
filtered_docs = []
for read in read1:
    tokens = word_tokenize(read)
    tmp = ""
    for w in tokens:
        if w not in sw:
            tmp += ps.stem(w)+ " "
    filtered_docs.append(tmp)
print(filtered_docs)

["' a bibliometr review waqf literatur hasan , r. jun 2022 ' , ", "' a note covid-19 instig maximum drawdown islam market versu convent counterpart hasan , r. may 2022 ' , ", "'bank stock valuat theori : explain price base theori ? zarei , a . 1 mar 2022 ' , ", "'ceo dualiti firm perform : a systemat review research agenda yu , m. 25 may 2022 ' , ", "'ceo financi experi firm ’ earn manag m & a : import experi specif nguyen , t. 7 mar 2022 ' , ", "'ceo tenur cost debt owusu-mensah , r. aug 2022 ' , ", "'cheat behaviour among opec member-st oil price fair stabil : an empir analysi ibrahim , m. 2022 ' , ", "'clinician ’ inform acquisit account literaci uk clinic commiss group enombu , j . a . 22 jun 2022 ' , ", "'compet institut logic power dynam islam financi report standardis project abra , a . 10 may 2022 ' , ", "'corpor govern ifsb standard-4 : evid islam bank bangladesh ullah , h. 18 jan 2022 ' , ", "'corpor hedg cost equiti capit ahm , h. 2022 ' , ", "'corpor social respons & firm e

In [9]:
filtered_test_docs = []
tokens = word_tokenize(read2)
tmp = ""
for w in tokens:
    if w not in sw:
        tmp += ps.stem(w) + " "
        filtered_test_docs.append(tmp)


word_seggregation = filtered_test_docs[len(filtered_test_docs)-1].split()


word_seggregation[0:10]

['a',
 'bibliometr',
 'review',
 'waqf',
 'literatur',
 'hasan',
 'r',
 'jun',
 '2022',
 'a']

In [10]:
vectorize= TfidfVectorizer()
response= vectorize.fit_transform(filtered_docs)

response_array = response.toarray()
words_set = vectorize.get_feature_names()

df_tf_idf = pd.DataFrame(response_array, columns = words_set)
df_tf_idf


/Users/gokul_08/Coventry_DS/Softwares/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,10,100,12,13,14,16,17,18,19,2020,...,whi,win,world,worldwid,yu,zapomocą,zarei,znaczeni,çalışma,üzerinden
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.251253,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.298748,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.320778,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.211730,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.170467,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.132313,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.238817,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0


In [11]:
#Indexing 
word_dict = {}
for item in word_seggregation:
    for i in range(count) :
        check = filtered_docs[i].lower()
        if item in check:
            if item not in word_dict:
                word_dict[item]   = []
            if item in word_dict:
                if (i+1) not in word_dict[item]:
                    word_dict[item].append(i+1)


In [17]:
print(word_dict)

{'a': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114], 'bibliometr': [1, 70], 'review': [1, 4, 70, 80], 'waqf': [1], 'literatur': [1], 'hasan': [1, 2, 19, 31, 35, 49, 51, 54, 55, 61, 70, 81], 'r': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 

In [13]:
#Queryprocessing

def Query_processing(query1):
    query= []
    query.append(query1)
    filtered_query = []
    for doc in query:
        tokens2 = word_tokenize(doc)
        tmp2 = ""
        for w in tokens2:
            if w not in sw:
                tmp2 += ps.stem(w)+ " "
        filtered_query.append(tmp2)

#print(filtered_query)

    vectorize2= TfidfVectorizer()
    response_query= vectorize2.fit_transform(filtered_query)
    response_query_array = response_query.toarray()
    query_words_set = vectorize2.get_feature_names()

    query_tf_idf = pd.DataFrame(response_query_array, columns = query_words_set)
    
    
    index_of_doc=[]

    for i in filtered_query[0].split():
        if i in word_dict.keys():
            res = word_dict[i] 
        
            index_of_doc.append(res)

    doc_index_list = [item for secondary in index_of_doc for item in secondary] 
    doc_index= set(doc_index_list) 

    tfidf=[]

    for r in doc_index:
        temp=0
        for w in filtered_query[0].split():
            if w in words_set:
                temp = temp +  df_tf_idf[w][r] * query_tf_idf[w][0]
        tfidf.append(temp)
    
    s_tfidf, s_doc_index = zip(*sorted(zip(tfidf, doc_index), reverse=True))
#     print(s_doc_index)
#     print(s_tfidf)
    choice_list = ["Index","Keywords"]
    print("Results related to "+ query1)
    for i in s_doc_index:
        a = output_list[i-1]
        #print(type(a))
        for key,values in a.items():
            #print(key)
            if str(key) not in choice_list:
                print(str(key) + " : " + str(values))

        print("--------------------------------------------------------------------------------------------------------")

        



In [21]:
query1 = input("Enter the query need to be searched : ")
Query_processing(query1)

Enter the query need to be searched : islamic
Results related to islamic
Author Link : https://pureportal.coventry.ac.uk/en/persons/ahmad-abras
Title Link : https://pureportal.coventry.ac.uk/en/publications/competing-institutional-logics-and-power-dynamics-in-islamic-fina
Author Name : Abras, A.
Year of publication : 10 May 2022
--------------------------------------------------------------------------------------------------------
Author Link : https://pureportal.coventry.ac.uk/en/persons/sarkar-kabir
Title Link : https://pureportal.coventry.ac.uk/en/publications/exploring-the-prospects-for-islamic-home-finance-in-the-uk-eviden
Author Name : Kabir, S.
Year of publication : 2020
--------------------------------------------------------------------------------------------------------
Author Link : https://pureportal.coventry.ac.uk/en/persons/sailesh-tanna
Title Link : https://pureportal.coventry.ac.uk/en/publications/testing-dividend-life-cycle-theory-in-the-islamic-and-conventiona
Autho

In [22]:
#Long query

query1 = input("Enter the query need to be searched : ")
Query_processing(query1)




Enter the query need to be searched : Diffusion theory, transnational antecedents and International Standards on Auditing adoption around the world
Results related to Diffusion theory, transnational antecedents and International Standards on Auditing adoption around the world
Author Link : https://pureportal.coventry.ac.uk/en/persons/ahmad-abras
Title Link : https://pureportal.coventry.ac.uk/en/publications/competing-institutional-logics-and-power-dynamics-in-islamic-fina
Author Name : Abras, A.
Year of publication : 10 May 2022
--------------------------------------------------------------------------------------------------------
Author Link : https://pureportal.coventry.ac.uk/en/persons/ibrahim-elmghaamez
Title Link : https://pureportal.coventry.ac.uk/en/publications/financial-market-consequences-of-early-adoption-of-international-
Author Name : Elmghaamez, I.
Year of publication : 21 Jul 2020
------------------------------------------------------------------------------------------

In [154]:
pip install schedule


Note: you may need to restart the kernel to use updated packages.


In [20]:
import schedule

schedule.every(168).hours.do(Web_crawlerFunc)

Every 168 hours do Web_crawlerFunc() (last run: [never], next run: 2022-08-12 14:26:33)